## FLEXtopo impermeability code

In [ ]:
import numpy as np
from Weigfun import Weigfun

def urban(timestep, Par, forcing, Fluxes, States):
	# HBVpareto Calculates values of 3 objective functions for HBV model

	Imax = Par[0] #in this case this would be interception on buildings and streets
	Kf = Par[5] # I don't know if we should keep the original number of the paramenter of if it's ok to change it to 1

	Qo = forcing['Qo']
	Prec = forcing['Prec']
	Etp = forcing['Etp']


	tmax = len(Prec)
	Si = States[:,0]
	Su = States[:,1]
	Sf = States[:,2]

	Eidt = Fluxes[:,0]
	Eadt = Fluxes[:,1]
	Qfdt = Fluxes[:,2]
	Qusdt = Fluxes[:,3]

	dt = 1
	t = timestep


	Pdt = Prec[t] * dt
	Epdt = Etp[t] * dt
	# Interception Reservoir
	if Pdt > 0:
		Si[t] = Si[t] + Pdt
		Pedt = max(0, Si[t]-Imax)
		Si[t] = Si[t] - Pedt
		Eidt[t] = 0
	else:
	# Evaporation only when there is no rainfall
		Pedt = 0
		Eidt[t] = min(Epdt, Si[t])
		Si[t] = Si[t] - Eidt[t]

	if t < tmax - 1:
		Si[t+1] = Si[t]


	# Fast Reservoir, DOES IT MAKE SENSE TO KEEP IT?
	Sf[t] = Sf[t] + Pedt
	Qfdt[t] = dt * Kf * Sf[t]
	Sf[t] = Sf[t] - min(Qfdt[t], Sf[t])
	if t < tmax - 1:
		Sf[t+1] = Sf[t]
	    
		
	# Save output
	States[:,0] = Si
	States[:,1] = 0
	States[:,2] = Sf

	Fluxes[:,0] = Eidt
	Fluxes[:,1] = 0
	Fluxes[:,2] = Qfdt
	Fluxes[:,3] = 0

	return(Fluxes, States)

    